# Notebook regarding the Final Project of Big Data Tools and Techniques.

- Rowan University - Fall 2025
- Student's name: **Luciano Stork (916547334)**
- Professor: Dr. Silvija Kokalj-Filipovic


The purpose of this work is to visually present how rural public schools with data connections, categorized by access technology and service provider, are distributed across the national territory. To this end, I will use Neo4J Database to illustrate this distribution based on the Brazilian states.


## 1) Data Collection:

Note:
- The file "Relacao_Escolas_Rurais_Atendidas.csv" was originally collected from the official website of the Brazilian National Telecommunications Agency (Anatel) via the following URL: https://dados.gov.br/dados/conjuntos-dados/escolas-rurais-conectadas


- However, security limitations prevented me from obtaining the direct download URL from the aforementioned site. As a solution, I uploaded the file to my personal Cloud Drive and from there I was able to download it to my local machine.

 * Link to access the file via my Cloud Drive: https://drive.google.com/file/d/1FFW_zLlqF8W7RX166bapVM0IH7fe3PKD/view?usp=sharing

In [1]:
# first, install gdown if you haven't already
# !pip install gdown

import gdown # type: ignore

# shared file link
file_id = "1FFW_zLlqF8W7RX166bapVMpIH7fe3PKD"
url = f"https://drive.google.com/uc?id={file_id}"

# local file name
output = "Relacao_Escolas_Rurais_Atendidas.csv"

# download the file
gdown.download(url, output, quiet=False)

print("Download completed!")


Downloading...
From: https://drive.google.com/uc?id=1FFW_zLlqF8W7RX166bapVMpIH7fe3PKD
To: /Users/desouz78/Documents/studying/Big Data Tools and Techniques/FinalProject/Relacao_Escolas_Rurais_Atendidas.csv
100%|██████████| 12.2M/12.2M [00:11<00:00, 1.04MB/s]

Download completed!


## 2) Visualize the data 

Note:
- Given that this is a database collected and processed by a Brazilian federal organization, it is reasonable to assume that the data received considerable attention in terms of prior confirmation and verification before being shared. This explains its consistency, as there are no duplicate, empty, or inconsistent entries.

- Therefore, on my part, no data processing was necessary beyond a visual and logical review of the records.




In [7]:
# Install pandas if you haven't yet
#!pip install pandas

import pandas as pd

# Load the CSV
csv_file = "Relacao_Escolas_Rurais_Atendidas.csv"
df = pd.read_csv(csv_file, encoding="latin1", sep=";")

# Optional: limit display widths to avoid line breaks
pd.set_option('display.max_columns', None)     # show all columns
pd.set_option('display.max_colwidth', 25)     # limit width of each column
pd.set_option('display.width', 150)           # total display width
pd.set_option('display.expand_frame_repr', False)  # prevent line wrapping

# 1️⃣ Basic overview
print("=== Basic Info ===")
print(df.info())        # data types, non-null counts
print("\n=== First 5 rows ===")
print(df.head())        # first rows
print("\n=== Shape ===")
print(f"Rows: {df.shape[0]}, Columns: {df.shape[1]}")

# 2️⃣ Check for missing values
print("\n=== Missing values per column ===")
print(df.isnull().sum())

# 3️⃣ Check for duplicate rows
num_duplicates = df.duplicated().sum()
print(f"\nNumber of duplicate rows: {num_duplicates}")

# 4️⃣ Check for inconsistent or suspicious data
print("\n=== Unique values per column ===")
for col in df.columns:
    unique_vals = df[col].nunique()
    print(f"{col}: {unique_vals} unique values")


=== Basic Info ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29012 entries, 0 to 29011
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   NomePrestadora      29012 non-null  object
 1   CodINEP             29012 non-null  object
 2   siglaUf             29012 non-null  object
 3   NomeMunicipio       29012 non-null  object
 4   DescSituacao        29012 non-null  object
 5   TipoObrigacao       29012 non-null  object
 6   VelAcessoInstalada  29012 non-null  object
 7   DescTecnologia      29012 non-null  object
 8   NomeEscola          29012 non-null  object
 9   DescTipoEscola      29012 non-null  object
 10  DataAtivacao        29012 non-null  object
dtypes: object(11)
memory usage: 2.4+ MB
None

=== First 5 rows ===
  NomePrestadora      CodINEP siglaUf       NomeMunicipio DescSituacao             TipoObrigacao VelAcessoInstalada DescTecnologia                NomeEscola DescTipoEscola DataAtiv

## 3) Connect with Neo4J Database and Upload Data:

In [6]:
import pandas as pd
from neo4j import GraphDatabase

# Load the CSV
csv_file = "Relacao_Escolas_Rurais_Atendidas.csv"
df = pd.read_csv(csv_file, sep=";", encoding="latin1")

# Strip whitespace from string columns
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# Connect to Neo4j Aura
uri = "neo4j+s://b6992aaa.databases.neo4j.io"
user = "neo4j"
password = "Ll0OGnyjEEGGdr6N8XWuDM-eoxERJjNmQP7kZHH1JCk"
driver = GraphDatabase.driver(uri, auth=(user, password))

# Function to insert one row as nodes + relationships
def insert_row(tx, row):

    # Merge State
    tx.run(
        """
        MERGE (s:State {sigla: $siglaUf})
        """,
        siglaUf=row["siglaUf"]
    )

    # Merge City with composite identity (city + state)
    tx.run(
        """
        MERGE (c:City {
            name: $city,
            state: $siglaUf
        })
        WITH c
        MATCH (s:State {sigla: $siglaUf})
        MERGE (s)-[:HAS_CITY]->(c)
        """,
        city=row["NomeMunicipio"],
        siglaUf=row["siglaUf"]
    )

    # Merge School using CodINEP as unique identifier
    tx.run(
        """
        MERGE (sch:School {CodINEP: $CodINEP})
        SET sch.name = $school_name,
            sch.DescTipoEscola = $DescTipoEscola
        WITH sch
        MATCH (c:City {
            name: $city,
            state: $siglaUf
        })
        MERGE (c)-[:HAS_SCHOOL]->(sch)
        """,
        CodINEP=row["CodINEP"],
        school_name=row["NomeEscola"],
        DescTipoEscola=row["DescTipoEscola"],
        city=row["NomeMunicipio"],
        siglaUf=row["siglaUf"]
    )

    # Merge Technology and relationship with properties
    tx.run(
        """
        MERGE (tech:Technology {name: $technology})
        WITH tech
        MATCH (sch:School {CodINEP: $CodINEP})
        MERGE (sch)-[r:USES_TECHNOLOGY]->(tech)
        SET r.DescSituacao = $DescSituacao,
            r.TipoObrigacao = $TipoObrigacao,
            r.VelAcessoInstalada = $VelAcessoInstalada,
            r.NomePrestadora = $NomePrestadora,
            r.DataAtivacao = $DataAtivacao
        """,
        technology=row["DescTecnologia"],
        CodINEP=row["CodINEP"],
        DescSituacao=row["DescSituacao"],
        TipoObrigacao=row["TipoObrigacao"],
        VelAcessoInstalada=row["VelAcessoInstalada"],
        NomePrestadora=row["NomePrestadora"],
        DataAtivacao=row["DataAtivacao"]
    )

# Upload all rows
with driver.session() as session:
    for _, row in df.iterrows():
        session.execute_write(insert_row, row)

print("All rows uploaded successfully!")

/var/folders/c6/cnsw28bs7j34_n66kkvx0qhw0000gq/T/ipykernel_27272/3109590119.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


All rows uploaded successfully!
